In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

In [ ]:
df = pd.read_csv("/content/lc_loan_clean_data.csv")

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d']) #error in file
df.shape #(841943, 24)

In [ ]:
df.info()

# Descriptive Analysis
First, we will add two featured columns to enhance the analysis:

a) The repayment ratio is defined as the total amount paid back divided by the original loan amount.

b) Risk_flag is to flag a loan as high risk if either it has a problematic loan status or a DTI above 30%:

In [ ]:
df['repayment_ratio'] = df['total_pymnt'] / df['funded_amnt']
df['repayment_ratio'] = df['repayment_ratio'].replace([np.inf, -np.inf], 0)

In [ ]:
df['repayment_ratio'].describe()

In [ ]:
high_risk_statuses = ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)']
threshold = 30
df['risk_flag'] = np.where(
    df['loan_status'].isin(high_risk_statuses) | (df['dti'] > threshold),
    1,
    0
)

In [ ]:
df['risk_flag'].value_counts()

How is it different the repayment_ratio depending on the risk flag?

In [ ]:
risk_repayment_summary = df.groupby('risk_flag')['repayment_ratio'].agg(['min', 'max', 'mean', 'std'])
print(risk_repayment_summary)

# Risk segmentation table

In [ ]:
status_by_risk = df.groupby('risk_flag')['loan_status'].value_counts(normalize=True).unstack(fill_value=0)
status_by_risk

Careful with the interpretation, remember that we defined high risk as:
high_risk_statuses = ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)'] or if the borrower's debt-to-income ratio (DTI) is above 30%.

Then, loans with risk_flag=0 are guaranteed not to have any of these negative statuses, and that is why there is a 0% for "Charged Off" and “Default” in that group.
However, this table gives us the distribution of the no-null values and how the dti ratio plays a role in risk segmentation.

# Loan status vs repayment ratio

In [ ]:
repayment_summary = df.groupby('loan_status')['repayment_ratio'].agg(['count', 'mean'])
repayment_summary = repayment_summary.sort_values(by='count', ascending=False)
print (repayment_summary)

It is important to note that loans classified as "Current" or "Issued" have lower repayment ratios because their repayment cycle is still in progress. As a result, calculating the average of repayment ratio across the entire dataset can be misleading. These incomplete loans skew the average downward and do not accurately reflect the complete repayment behavior of loans that have finished their term.

# Distribution of grade

In [ ]:
grade_summary = df.groupby('grade')['int_rate'].agg(['min', 'max', 'mean','count']).reset_index()

print(grade_summary)

# time series

In [ ]:
df2008 = df[(df["issue_d"] >= "2008-01-01") & (df["issue_d"] < "2009-01-01")]
df2008['funded_amnt'].sum()

In [ ]:
df2009 = df[(df["issue_d"] >= "2009-01-01") & (df["issue_d"] < "2010-01-01")]
df2009['funded_amnt'].sum()

In [ ]:
51722950/19917425 # verifying the following table (2.59 - 1)*100 is the percentage increase

In [ ]:
# Extract the year from the 'issue_d' column
df['year'] = df['issue_d'].dt.year

# Group by year and sum the 'funded_amnt'
yearly_funded_amnt = df.groupby('year')['funded_amnt'].sum()

# Convert the result to a DataFrame:
yearly_funded_amnt_df = yearly_funded_amnt.reset_index()
yearly_funded_amnt_df.columns = ['Year', 'Total Funded Amount']

# Calculate YoY increase
yearly_funded_amnt_df['YoY Increase'] = yearly_funded_amnt_df['Total Funded Amount'].pct_change() * 100

# Replace NaN (for the first year)
yearly_funded_amnt_df['YoY Increase'] = yearly_funded_amnt_df['YoY Increase'].fillna(0)

# Convert number type
yearly_funded_amnt_df['Total Funded Amount'] = yearly_funded_amnt_df['Total Funded Amount'].astype(int)
yearly_funded_amnt_df['YoY Increase'] = yearly_funded_amnt_df['YoY Increase'].round(2)

# Print the DataFrame
print(yearly_funded_amnt_df)

# Return on Investment

In [ ]:
roi_selected = df[["funded_amnt", "term", "installment", "loan_status", "total_pymnt", "total_rec_int", "total_rec_prncp","repayment_ratio","risk_flag"]]

In [ ]:
roi_selected.info()

In [ ]:
roi_selected['verify'] = roi_selected['total_pymnt'] - roi_selected['total_rec_int'] - roi_selected['total_rec_prncp']

In [ ]:
roi_selected_sorted = roi_selected.sort_values(by='verify', ascending=False)
roi_selected_sorted


Conclusion: In many cases, the total payment does not match the sum of the recovered principal and interest. This discrepancy may be due to clients receiving alternative settlement offers or adjustments from the bank at the time of loan closure.
For the ROI analysis, we will assume the total payment column is correct.

In [ ]:
completed_loans = roi_selected[
    roi_selected['loan_status'].isin([
        'Fully Paid',
        'Charged Off',
        'Default',
        'Does not meet the credit policy. Status:Fully Paid',
        'Does not meet the credit policy. Status:Charged Off'
    ])
]

In [ ]:
completed_loans.info()

In [ ]:
# Filter data by term
term_36 = completed_loans[completed_loans['term'] == 36]
term_60 = completed_loans[completed_loans['term'] == 60]

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
table_36 = term_36.groupby('loan_status').agg({
    'funded_amnt': 'sum',
    'total_pymnt': 'sum',
    'repayment_ratio': 'mean',
    'installment': 'count'
}).reset_index()

table_36.columns = ['Loan Status', 'Total Funded Amount', 'Total Payment Amount', 'Average Repayment Ratio', 'Loan Count']

table_36

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
table_60 = term_60.groupby('loan_status').agg({
    'funded_amnt': 'sum',
    'total_pymnt': 'sum',
    'repayment_ratio': 'mean',
    'installment': 'count'
}).reset_index()

table_60.columns = ['Loan Status', 'Total Funded Amount', 'Total Payment Amount', 'Average Repayment Ratio', 'Loan Count']

table_60

In [ ]:
roi_36 = 100*(table_36['Total Payment Amount'].sum() / table_36['Total Funded Amount'].sum())-100
print(f"The Return on Investment for 36-month closed loans is {roi_36:.2f}%")

In [ ]:
roi_60 = 100*(table_60['Total Payment Amount'].sum() / table_60['Total Funded Amount'].sum())-100
print(f"The Return on Investment for 60-month closed loans is {roi_60:.2f}%")

# Interesting Visuals

In [ ]:
risk_counts = pd.Series({0: 751376, 1: 135967})
plt.figure(figsize=(8, 6))
risk_counts.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title("Number of Clients by Risk Flag")
plt.xlabel("Risk Flag (0 = Lower, 1 = Higher)")
plt.ylabel("Number of Clients")
plt.xticks(rotation=0)
plt.show()

In [ ]:
mean_repayment = pd.Series({0: 0.562273, 1: 0.406078})
plt.figure(figsize=(8, 6))
mean_repayment.plot(kind='bar', color=['green', 'orange'])
plt.title("Mean Repayment Ratio by Risk Flag")
plt.xlabel("Risk Flag (0 = Lower, 1 = Higher)")
plt.ylabel("Mean Repayment Ratio")
plt.xticks(rotation=0)
plt.show()

In [ ]:
data = {'Charged Off': [0.000000, 0.332779],
        'Current': [0.715008, 0.474483],
        'Default': [0.000000, 0.008965],
        'Does not meet the credit policy. Status:Charged Off': [0.001013, 0.000000],
        'Does not meet the credit policy. Status:Fully Paid': [0.00264, 0.000000],
        'Fully Paid': [0.264291, 0.067193],
        'In Grace Period': [0.007316, 0.005553],
        'Issued': [0.009731, 0.008443],
        'Late (16-30 days)': [0.000000, 0.017335],
        'Late (31-120 days)': [0.000000, 0.085249]}
df1 = pd.DataFrame(data, index=[0, 1])


df1.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Loan Status Distribution by Risk Flag')
plt.xlabel('Risk Flag')
plt.ylabel('Percentage')
plt.xticks(rotation=0)
plt.legend(title='Loan Status', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
grade_data = {
    'grade': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    'min': [5.32, 6.00, 6.00, 6.00, 6.00, 6.00, 16.59],
    'max': [9.63, 14.09, 17.27, 20.31, 23.40, 26.06, 28.99],
    'mean': [7.243304, 10.82967, 13.980123, 17.17584, 19.897326, 23.582787, 25.626706],
    'count': [148187, 254525, 245854, 139539, 70703, 23046, 5489]
}
grade_df = pd.DataFrame(grade_data)
plt.figure(figsize=(8, 6))
sns.barplot(x='grade', y='count', data=grade_df, palette="viridis", hue='grade', legend=False)
plt.title("Number of Loans by Grade")
plt.xlabel("Grade")
plt.ylabel("Loan Count")
plt.show()

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'])
plt.figure(figsize=(12, 6))
df['issue_month'] = df['issue_d'].dt.to_period('M')
loans_by_month = df.groupby('issue_month').size()
loans_by_month.plot(kind='line', marker='o')
plt.title("Loan Issuance Trend Over Time")
plt.xlabel("Month")
plt.ylabel("Number of Loans Issued")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(status_by_risk, annot=True, cmap="YlGnBu", fmt=".3f")
plt.title("Heatmap of Loan Status Proportions by Risk Flag")
plt.xlabel("Loan Status")
plt.ylabel("Risk Flag")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='term', hue='loan_status', data=df, palette="Set1")
plt.title("Loan Term vs. Loan Status")
plt.xlabel("Term (in months)")
plt.ylabel("Count")
plt.legend(title="Loan Status", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(df['int_rate'], bins=30, kde=True, color='teal')
plt.title("Distribution of Interest Rates")
plt.xlabel("Interest Rate")
plt.ylabel("Frequency")
plt.show()

Kernel Density Estimate (KDE):

The KDE line is a smooth curve that estimates the probability density function of the interest rate data, it provides a continuous estimate of how the data is distributed across the range of interest rates.

Conclusion: there are more loans with low interest rates.

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['loan_amnt'], bins=50, kde=True, color='blue')
plt.title("Distribution of Loan Amount")
plt.xlabel("Loan Amount")
plt.ylabel("Frequency")
plt.show()

The part on the left represents the "Issued" and "Current" loans, while the one on the right represents the "Fully Paid".

In [ ]:
plt.figure(figsize=(10, 6))
upper_limit = df['annual_inc'].quantile(0.99)
df_filtered = df[df['annual_inc'] <= upper_limit]

sns.histplot(df_filtered['annual_inc'], bins=40, kde=True, color='blue')
plt.title("Distribution of Annual Income")
plt.xlabel("Annual Income")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y='purpose', data=df, order=df['purpose'].value_counts().index, palette='viridis', hue='purpose', legend=False)
plt.title("Distribution of Loan Purpose")
plt.xlabel("Count")
plt.ylabel("Loan Purpose")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='addr_state', data=df,
              order=df['addr_state'].value_counts().index,
              hue='addr_state',
              palette='viridis',
              legend=False)
plt.title("Loan Issuance by State")
plt.xlabel("State")
plt.ylabel("Number of Loans")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
df.to_csv("lc_loan_to_diagnostic.csv", index= False)

# Data for Tableau

In [ ]:
numeric_cols = ['loan_amnt', 'funded_amnt', 'int_rate', 'installment',
                'annual_inc', 'dti', 'total_pymnt', 'total_rec_int',
                'total_rec_prncp', 'delinq_2yrs', 'open_acc', 'total_acc',
                'revol_util', 'emp_length_int']
corr_matrix = df[numeric_cols].corr()


In [ ]:
# Reset index to turn row labels into a column
corr_df = corr_matrix.reset_index().rename(columns={'index': 'Variable1'})
# Melt the data frame to convert from wide to long format
corr_long = pd.melt(corr_df, id_vars='Variable1', var_name='Variable2', value_name='Correlation')
corr_long.to_csv("corr_long.csv", index=False)

In [ ]:
data = {
    'Charged Off': [0.000000, 0.332779],
    'Current': [0.715008, 0.474483],
    'Default': [0.000000, 0.008965],
    'Does not meet the credit policy. Status:Charged Off': [0.001013, 0.000000],
    'Does not meet the credit policy. Status:Fully Paid': [0.00264, 0.000000],
    'Fully Paid': [0.264291, 0.067193],
    'In Grace Period': [0.007316, 0.005553],
    'Issued': [0.009731, 0.008443],
    'Late (16-30 days)': [0.000000, 0.017335],
    'Late (31-120 days)': [0.000000, 0.085249]
}
df1 = pd.DataFrame(data, index=[0, 1]).reset_index().rename(columns={'index': 'risk_flag'})
# Pivot the dataframe from wide to long format using melt
df1_long = pd.melt(df1, id_vars='risk_flag', var_name='Loan_Status', value_name='Percentage')
df1_long.to_csv("loan_status_distribution.csv", index=False)
